In [21]:
import math
import mysql.connector as mysql
import numpy as np
import os
import pandas as pd
import re
import urllib.request

from os.path import splitext
from pathlib import Path
from PIL import ImageFile

In [22]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

# == Files config ==
FILE_FOLDER = r"D:\legal_data\marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv"
fdata=pd.read_csv(Path(FILE_FOLDER))
IMG_FOLDER_DEST = r"D:\images\amazon"

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_amazon"

In [23]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_amazon")

In [24]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_amazon ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [25]:
fdata.columns

Index(['Uniq Id', 'Product Name', 'Brand Name', 'Asin', 'Category',
       'Upc Ean Code', 'List Price', 'Selling Price', 'Quantity',
       'Model Number', 'About Product', 'Product Specification',
       'Technical Details', 'Shipping Weight', 'Product Dimensions', 'Image',
       'Variants', 'Sku', 'Product Url', 'Stock', 'Product Details',
       'Dimensions', 'Color', 'Ingredients', 'Direction To Use',
       'Is Amazon Seller', 'Size Quantity Variant', 'Product Description'],
      dtype='object')

In [26]:
# Deleting unwanted columns
del_list = ['Uniq Id', 'Brand Name', 'Asin', 'Upc Ean Code', 'List Price', 'Quantity', 'Technical Details', 
            'Model Number', 'Shipping Weight', 'Product Dimensions', 'Variants', 'Sku', 'Product Url', 'Stock', 
            'Product Details', 'Dimensions', 'Color', 'Ingredients', 'Direction To Use', 'Is Amazon Seller', 
            'Size Quantity Variant', 'Product Description']

fdata = fdata.drop(del_list, axis = 1)

fdata.columns

Index(['Product Name', 'Category', 'Selling Price', 'About Product',
       'Product Specification', 'Image'],
      dtype='object')

In [27]:
def update_image(product_id, value):
    query = "UPDATE products_tmp_amazon SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [28]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()
    
    return clear_content

In [29]:
def get_ext(url):
    filename, file_extension = os.path.splitext(url)
    
    return file_extension

In [30]:
def insert_product(row):
    price = "" if len(row['Selling Price']) > 50 else row['Selling Price']
    
    encoded_string = row['About Product'].encode('utf-8', "ignore")
    description = encoded_string.decode()
    
    description = description.replace("Make sure this fits by entering your model number. | ", "")
    description = re.sub('[👍🌍📱✅【】👐🔬⚗️⚙️🧜😎🔴🔵🙌🌿🎁🦄💃👜⭐💦]', '', description)
    description = clear_text(description.replace(' | ', ''))

    print(description)
    
    query = "INSERT INTO products_tmp_amazon (name, description, price, market_name, has_image, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['Product Name'], description, price, "Amazon", "no_image", "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [31]:
for index, row in fdata.iterrows():
    if row['Product Name'] == "NaN" or type(row['Product Name']) is not str: row['Product Name'] = ""
    if row['About Product'] == "NaN" or type(row['About Product']) is not str: row['About Product'] = ""
    if row['Selling Price'] == "NaN" or type(row['Selling Price']) is not str: row['Selling Price'] = ""

    if row['Product Name'] is not "" and row['About Product'] is not "":
        product_id = insert_product(row)

        images = row['Image'].split(sep="|")
        img_index = 0

        for img in images:
            if not img == "https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg":
                img_ext = get_ext(img)

                folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}{img_ext}")

                if img_index > 0:
                    folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}_{img_index}{img_ext}")

                try:
                    file = urllib.request.urlretrieve(img, folder_dest)
                    img_index = img_index + 1
                except:
                    print(f"Product {product_id}: Error retrieving image ({img})")

                if img_index > 0 : update_image(product_id, "has_image")

RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern that’s second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds.COREFLEX TECH: CoreFlex construction is water resistant, impact resistant, scratch resistant and has a flex like you won’t believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength.INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the NorthwestBEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like familyBEYOND COMPARE: Try our skateboards & accessories if you've tried similar products by Sector 9, Landyach

Perfect all-inclusive resource for teachersProvides focus for instruction in the perfect time-saving formatPre-printed, easily organized system that provides an entire list of Common Core State Standards in both math and language artsIncludes divider cards, grade-specific 2-sided cards with "I Can" statements on one side and the corresponding standard on the other124 standards cards, 11 dividers
Collaborative Trapezoid Activity TableTable can be used alone or grouped together for cooperative learningRecommended Grade Level: PreschoolRecommended Seating Capacity: 3 ChildrenScratch and Stain Resistant Surface
For the animal lover in the family, a complete veterinarian kit; perfect for the aspiring vetContains everything you need, 8 specialist veterinarian tools, Clipboard, and 2 special x-ray cards that you can see through when held up to the lightA great STEM/ steam toy, learn about animals and their health needs, Features dogs, cats, rabbits, horses and a pot-bellied pigComes in a port

Product 37: Error retrieving image (https://images-na.ssl-images-amazon.com/images/I/B18UMISIHxS.jpg)
From persona 5, the Joker (styles may vary), as a stylized POP vinyl from Funko!Stylized collectable stands 3 ¾ inches tall, perfect for any persona 5 fan!There's a 1/6 chance you'll receive a rare, Chase version, of the Joker!Collect and display all persona 5 items from Funko!
A guided journal that you personalize inside and outLearn frosting-like techniquesHigh-quality puffy paste and fancy charms, plus Klutz clear instructions how to use themGreat for Ages 8+Includes a 60 pages of instructions and journal pages
Looking to instill a little fear in your Game? Then look no further!Nothing brings the thrill – and terror – of discovery to a Game like new monsters. Faced with the unknown, mighty-thewed heroes tremble in their hauberks, Wizen Wizards fumble with their spell books, and even the most audacious of rogues hesitate before plunging into battle.Nevermore worry that your players h

VEHICLE WITH LIGHTS AND SOUNDS: Chase’s cruiser features lights and sounds and caster wheels that make it possible to drive in any direction! Push the button to activate the lights and sounds!HOVERCRAFT MODE: With working caster wheels, it’s easier than ever to roll your Charged Up Deluxe Vehicle to the rescue. Spin and drift 360 degrees in any direction and help save the day!INCLUDES CHASE FIGURE: Wearing his blue Charged Up uniform, place the included Chase collectible figure behind the wheel of his deluxe cruiser. This pup hero is ready for anything that comes his way!The PAW Patrol Charged Up Chase Deluxe Vehicle makes a great gift for kids aged 3 and up. Requires 3 LR44 batteries (included). Get charged up with PAW Patrol and their Deluxe Vehicles!Includes: 1 Chase Figure, 1 Charged Up Deluxe Chase Vehicle
Match and snap to complete the pictureDevelops hand eye coordination, fine motor skills and creative thinkingConvenient storage tray holds all piecesIncludes 46 colorful buttons

Premium rolling stock for your N scale layoutNmra profile metal wheelsNeedle-point axlesBody-mounted E-Z Mate Mark II couplersN scale: 1: 160
This item is a great value!1 per packageGeneral Occasion party itemCutouts - Foil for festive occasionsHigh Quality
N/AImportedDeck measures 2.5 x 3.5Deck contains multiple imagesOfficially licensed merchandiseCards have a linen Type finish on them
Officially Licensed by Great Eastern!Makes a great gift!Cute and CollectibleLimited availabilityApprox Size: 5.5"L x 3.5"W x 9"H
Precision RC hobby partsCheck your users manual for exact parts listings
Tri-channel transmitter: Three people can play at the same time if they have their own Maisto R/C VehiclesDifferential gearboxRequires 6 AA batteries for the vehicle and 2 AAA batteries for the controller (Batteries Not Included)
Product 95: Error retrieving image (https://images-na.ssl-images-amazon.com/images/I/61YzVQ-lkWS.jpg)
52 different images per deckLinen type finishOfficially licensed merchandis

...

Each kit includes: 1 Foil Fringe Centerpiece, 12" 2 3-D Printed Paper Centerpieces, 7" 20 Pieces of Paper Confetti, 2"Made of paper and foilFeatures a sweet blue color scheme and eye-catching centerpieces with matching table scatter in the shape of a crossPerfect for first communion table decoration
Plastic24" high3" wideGray plastic Tin Man axeApproximately 24" longOfficially licensed Wizard of Oz accessoryAccessories make the costumeComplete your Tin Man costume
A new Villain Pack for Imperial AssaultSavagely hunt down your opponent's figures with a sculpted plastic figure of BosskA new campaign side mission invites you to smuggle guns through the Toydarian wildernessTwo new skirmish missions that share a map invite you to travel to the Wasskah Hunting GroundNew Agenda cards, Deployment cards, and Command cards bring changing tactics to every game
BANNER SIZE: 24" X 72"Indoor/Outdoor Use – Our Banners are durable, made from heavy duty 13oz. outdoor vinyl and our inks are rated for 5+

SET THE TREND: Show off your unique style with MightySkins for your Parrot Bebop 2! Don’t like the Black Zebra skin? We have hundreds of designs to choose from, so your Bebop 2 will be as unique as you are!DURABLE PROTECTION FOR YOUR DEVICE: Manufactured with ultra-thin, ultra-durable, stain-resistant laminate vinyl in order to effectively protect your gear from dings, scrapes, dust, and the wear and tear of everyday use!GOES ON AND COMES OFF EASY: Putting your MightySkins on your Bebop 2 is super easy thanks to its patented low grip air release adhesive that’s built to last but removes easily and leaves no sticky residue when you’re ready to switch to a different design!SATISFACTION : Our friendly, knowledgeable staff is here to help and customer satisfaction is our top priority! If you have any issues with your order or you’re not happy for any reason, just let us know and we’ll be there to help!PROUDLY MADE IN THE USA: We Use Only 3M automotive-grade vinyl, the industry's most trust

Modular plastic building blocks allow you to build the objects of your desirePerfect for all types of constructions and creationsEverblock oversized blocks are ideal for creative building projects and stem educationDurable plastic blocks are weather and chemical resistantCombo pack contains: 16 Full blocks, 4 half blocks, 6 finishing caps, add blocks as needed
ONE OF THE LARGEST MARINE PREDATORS - The great white shark is one of the largest marine predators and has very few competitors. It owes its name to its white belly which stands in stark contrast to its grey back, they are found in all temperate waters around the world, and mainly in shallow water.PART OF THE AQUATIC ANIMALS - Sea creatures collection is not complete without this White Shark figure. It has a highly developed sense of hearing and smell, making it a formidable hunter.GREAT EXPERIENCE FOR KIDS - This small, durable and look cool figure is the best toy to experience for your kid especially for those aquatic animals l

The 640 X 480 resolution video gives you the perfect balance of quality and low memory spaceCapture everything on the 1.8 inch TFT preview screenThe rugged sports design lets you easily handle the camera in the heat of the momentPerfect for people on-the-go
Complete your nursery in style with the Nautica Kids - Pink Heart shaped marquee lighted wall decor comes ready to hang.Measures 10" x 9.5" x 2.4". Has 19 LED lights.Made of metal with a matte enameled finish.Uses 2 AA Batteries (not included) with on/off switch and 2 hour daily timer.
Puffers Penguin plush with festive winter capSoft, huggable material built to famous GUND quality standardsSurface-washable10 inch height (25.4 cm)Appropiate for ages 1+
100% CottonMade in the USA and ImportedMachine WashRuns true to size100 percent licensed men's t-shirtCrew neckShort sleeves
Product 9699: Error retrieving image (https://images-na.ssl-images-amazon.com/images/I/516iN9jCWgL.jpg)
Product 9699: Error retrieving image (https://images-na.

100% cottonImportedQuilt: 66 inches wide x 86 inches longShams: 20 inches wide x 26 inches longSet includes: 1 quilt and 1 shamShell is 100Percent cotton, fill is 100Percent cotton
Large size lego storage binPlastic polypropylene, pvc freeGather all your regular lego bricks in this large size storage brick and make it a more playful way of tidying upThis large size LEGO storage brick also work as stackable blocks with the rest of the storage bricksStorage bricks come in classic lego colorsLarge size lego storage binPlastic - polypropylene (pp), pvc freeGather all your regular lego bricks in this large size storage brick and make it a more playful way of tidying upThis large size LEGO storage brick also work as stackable blocks with the rest of the storage bricksStorage bricks come in classic lego colors
Dimensions: 6.375''X3''x0.25''Strong, reliable 3M adhesive that allows for easy peel and stick application and repositioningWallet function carries cards or cashKeep cords organized wit